# Day 23: Amphipod

A group of amphipods notice your fancy submarine and flag you down. "With such an impressive shell," one amphipod says, "surely you can help us with a question that has stumped our best scientists."

They go on to explain that a group of timid, stubborn amphipods live in a nearby burrow. Four types of amphipods live there: Amber (A), Bronze (B), Copper (C), and Desert (D). They live in a burrow that consists of a hallway and four side rooms. The side rooms are initially full of amphipods, and the hallway is initially empty.

They give you a diagram of the situation (your puzzle input), including locations of each amphipod (A, B, C, or D, each of which is occupying an otherwise open space), walls (#), and open space (.).

The amphipods would like a method to organize every amphipod into side rooms so that each side room contains one type of amphipod and the types are sorted A-D going left to right.

## Data and Rules

Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of energy to move one step: Amber amphipods require 1 energy per step, Bronze amphipods require 10 energy, Copper amphipods require 100, and Desert ones require 1000. The amphipods would like you to find a way to organize the amphipods that requires the least total energy.

However, because they are timid and stubborn, the amphipods have some extra rules:

- Amphipods will never stop on the space immediately outside any room. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)
- Amphipods will never move from the hallway into a room unless that room is their destination room and that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)
- Once an amphipod stops moving in the hallway, it will stay in that spot until it can move into a room. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)

### Location

In [ ]:
static var doors = new int[] { 2, 4, 6, 8 };

record struct Location(
        int inRoom, // 0 == Hallway, 2 == Amber, 4 == Bronze, ...
        int pos) {
    public bool HomeFor(int colour)
        => inRoom == colour * 2;
    public bool Is(int isRoom, int isPos)
        => inRoom == isRoom && pos == isPos;
    public bool SameRoom(Location o)
        => o.inRoom == inRoom && inRoom > 0;
    public bool RoomBlock(Location o)
        => SameRoom(o) && o.pos <= pos;
    public bool Between(Location from, Location to) {
        if (from.RoomBlock(this) || to.RoomBlock(this)) return true;
        if (inRoom != 0) return false;

        var start = from.HallPos;
        var end = to.HallPos;

        return start < end 
            ? (start <= pos && pos <= end)
            : (start >= pos && pos >= end);
    }
    public int Steps(Location to)
        => StepsTo(to.HallPos, to.RoomPos);
    public int StepsTo(int end, int pos) {
        var start = HallPos;
        var depth = RoomPos + pos;
    
        return (start < end
                ? end - start
                : start - end
            ) + depth;
    }
    public bool InHall { get; } = inRoom == 0;
    public int HallPos { get; } = inRoom > 0 ? inRoom : pos;
    public int RoomPos { get; } = inRoom > 0 ? pos : 0;

    public override string ToString()
        => $"{inRoom}/{pos}";
    public Location Up()
        => new Location(inRoom, pos - 1);
}

static var hallLocs = Enumerable.Range(0, 11)
    .Where(i => !doors.Contains(i))
    .Select(i => new Location(0, i))
    .ToArray();

static var roomDepth = 2;

### Amphipod

In [ ]:
record struct Amphipod(int colour, Location loc) {
    public bool Is(int isColour, int isRoom, int isPos)
        => colour == isColour && loc.Is(isRoom, isPos);
    public bool Home { get; } = loc.HomeFor(colour);
    public bool InHall { get; } = loc.InHall;
    public int Cost { get; } 
        = colour switch { 1 => 1, 2 => 10, 3 => 100, 4 => 1000, _ => 0 };
    public string Label { get; }
        = colour switch { 1 => "A", 2 => "B", 3 => "C", 4 => "D", _ => "." };
    public bool RoomMismatch(Amphipod a)
        => a.colour != colour && loc.SameRoom(a.loc);
    public bool BlockOut(Location l)
        => loc.RoomBlock(l);
    public bool BlockOut(Amphipod a)
        => BlockOut(a.loc);
    public bool BlockIn(Location l)
        => l.RoomBlock(loc);
    public bool Between(Location from, Location to)
        => loc.Between(from, to);
    public int Steps(Location to)
        => loc.Steps(to);
    public int Energy()
        => Home ? 0 : loc.StepsTo(colour * 2, 1) * Cost;
    public override string ToString() => $"{Label} @{loc}";
}

In [ ]:
var pod = new Amphipod(3, new Location(6, 1));
pod

colour,loc,Home,InHall,Cost,Label
3,"{ 6/1: inRoom: 6, pos: 1, InHall: False, HallPos: 6, RoomPos: 1 }",True,False,100,C


## Processing


### Action

In [ ]:
class Action {
    public Action(Amphipod actor, Location to) {
        Actor = actor;
        To = to;

        Cost = Actor.Steps(To) * Actor.Cost;
        Energy = Actor.Cost * To.StepsTo(Actor.colour * 2, 1);
    }

    public Amphipod Actor { get; }
    public Location To { get; }
    public int Cost { get; }
    public int Energy { get; }

    public bool IsTo(int isColour, int isRoom, int isPos)
        => Actor.colour == isColour && To.Is(isRoom, isPos);

    public bool Blocked(Amphipod block)
        => Actor.BlockOut(block) || block.BlockIn(To) || block.Between(Actor.loc, To);
    public bool Blocked(Location by)
        => Actor.BlockOut(by) || by.RoomBlock(To) || by.Between(Actor.loc, To);
    public int[] UsesHalls() {
        var start = Actor.loc.HallPos;
        var end = To.HallPos;

        return doors.Concat(start < end 
                ? Enumerable.Range(start, end-start+1)
                : Enumerable.Range(end, start-end+1)
            ).Distinct().ToArray();
    }
    public override string ToString() => $"{Actor} >{To} ${Cost}/{Energy}";
}

### State

In [ ]:
class State {
    public State(Amphipod[] current, Action[] taken) {
        Current = current ?? Array.Empty<Amphipod>();
        Taken = taken ?? Array.Empty<Action>();

        Key = new string(MakeRooms().SelectMany(r => r).ToArray());
        Cost = Taken.Sum(a => a.Cost);
        Energy = Current.Sum(a => a.Energy());
        Finished = Current.All(a => a.Home);
    }

    public Amphipod[] Current { get; }
    public Action[] Taken { get; }
    public string Key { get; }
    public bool Finished { get; }
    public int Energy { get; }
    public int Cost { get; }

    public bool Valid(Action act) {
        var actor = act.Actor;
        var others = Current.Where(c => c != actor); // || (c.colour == actor.colour && c.loc.pos == 1)).ToArray();
        if (act.To.HomeFor(actor.colour)) {
            others = others.Where(o => o.colour != actor.colour || o.loc.pos == 1);
        }
        return !others.Any(o => act.Blocked(o));
    }
    public Amphipod[] Blocks(Action act)
        => Current.Where(a => a.colour != act.Actor.colour && act.Blocked(a)).ToArray();

    public Action[] Split(Action act, Amphipod a) {
        if (a.InHall) return new[] { new Action(a, new Location(a.colour * 2, roomDepth)) };
        return hallLocs.Select(l => new Action(a, l)).ToArray();
        // Amphipod? inHall = Current.FirstOrDefault(i => i != a && i.loc.InHall);
        // if (inHall is Amphipod blocker) {
        //     splits = splits.Where(l => !l.Blocked(blocker.loc));
        // }
        // return splits.ToArray();
    }
    public State Perform(Action act) {
        if (Finished) return this;

        var inPlace = Current.Where(c => c.loc == act.To).ToArray();
        while (act.To.RoomPos > 1 && inPlace.Any()) {
            // Console.WriteLine($"{act} blocked by {inPlace[0]}");
            act = new Action(act.Actor, act.To.Up());
            inPlace = Current.Where(c => c.loc == act.To).ToArray();
        }

        var current = Current
            .Select(c => c == act.Actor ? new Amphipod(c.colour, act.To) : c);

        return new State(current.ToArray(), Taken.Append(act).ToArray());
    }
    public Action[] Needed()
        => Current.Where(c => !c.Home)
            .Select(a => new Action(a, new Location(a.colour * 2, roomDepth)))
            .ToArray();

    const char empty = ',';
    char[][] MakeRooms() {
        var r = Enumerable.Range(1, roomDepth+1).Select(i => new char[11]).ToArray();
        foreach (var e in r) {
            Array.Fill(e, ' ');
            e[2] = empty; e[4] = empty; e[6] = empty; e[8] = empty;
        }
        Array.Fill(r[0], empty);
        foreach (var a in Current) {
            char c = (char)('A' + a.colour - 1);
            var l = a.loc;
            if (l.inRoom > 0 && l.pos > roomDepth || l.inRoom == 0 && l.pos > 10)
                Console.WriteLine($"Invalid current {a}");
            else if (l.inRoom == 0) r[0][l.pos] = c;
            else r[l.pos][l.inRoom] = c;
        }
        return r;
    }

    public void ShowCurrent() {
        foreach (var e in MakeRooms()) {
            Console.WriteLine(e);
        }
        foreach (var t in Taken.Chunk(5)) {
            Console.WriteLine(string.Join(" | ", t.Select(a => a.ToString()).Prepend("")));
        }
        Console.WriteLine($"Energy : {Energy}  Cost: {Cost}");
    }
    public bool Equivalent(State other)
        => other != null 
            && Current.SequenceEqual(other.Current)
            && Cost == other.Cost;
}

### Tests

In [ ]:
var s = new State(new [] { new Amphipod(3, new Location(6, 2)), new Amphipod(3, new Location(6, 1))  }, null);
s.ShowCurrent();
s.Needed()

,,,,,,,,,,,
  , , C ,  
  , , C ,  
Energy : 0  Cost: 0


(empty)

In [ ]:
var blocker = new Amphipod(4, new Location(8, 1));
var s = new State(new[] { new Amphipod(1, new Location(8, 2)), new Amphipod(4, new Location(0, 5)), blocker }, null);

s.ShowCurrent();
display((s.Finished, s.Needed().Select(n => $"{n}")));

var act = s.Needed()[1];
display($"{act} ! {blocker}");
var blocks = s.Blocks(act);
blocks.Select(p => $"blocker {p}")


,,,,,D,,,,,
  , , , D  
  , , , A  
Energy : 4009  Cost: 0


Item1,Item2
False,"[ A @8/2 >2/2 $10/3, D @0/5 >8/2 $5000/3000 ]"


D @0/5 >8/2 $5000/3000 ! D @8/1

index,value
0,blocker A @8/2


## Attempts

In [ ]:
IEnumerable<State> Attempt(State curr) {
    foreach (var act in curr.Needed()) {
        if (curr.Valid(act)) {
            yield return curr.Perform(act);
        } else {
            foreach (var blocker in curr.Blocks(act)) {
                var split = curr.Split(act, blocker);

                foreach (var s in split) {
                    if (curr.Valid(s)) {
                        yield return curr.Perform(s);
                    }
                }
            }
        }
    }
}

In [ ]:
State Minimum(State curr) {
    var iteration = 1;
    HashSet<string> tried = new();
    HashSet<string> toTry = new();
    Dictionary<string,State> pending = new();
    State[] attempts;
    string key = curr.Key;
    pending[key] = curr;
    toTry.Add(key);

    while (pending.Count() > 0) {
        if (++iteration % 1000 == 0)  {
            Console.Write($"{iteration}: {pending.Count()}  ");
            if (iteration % 9000 == 0) Console.WriteLine();
        }

        attempts = Attempt(curr).ToArray();
        curr = pending.Values.MinBy(s => s.Cost);
        foreach (var a in attempts) {
            key = a.Key;
            if (tried.Contains(key)) continue;
            if (toTry.Contains(key)) {
                if (a.Cost >= pending[key].Cost) continue;
            } else
                toTry.Add(key);
            pending[key] = a;
            if (a.Cost < curr.Cost)
                curr = a;
        }

        if (curr.Finished) return curr;

        key = curr.Key;
        tried.Add(key);
        toTry.Remove(key);
        pending.Remove(key);
    }
    return null;
}

In [ ]:
State[] Guided(State curr, (int,int,int)[] expected) {
    State[] next;
    State[] distinct = new[] { curr };
    List<State> attempts = new();

    foreach (var (c, r, p) in expected) {
        attempts.Clear();
        foreach (var d in distinct) {
            attempts.AddRange(Attempt(d));
        }

        next = attempts.Where(n => n.Taken.Any(t => t.IsTo(c, r, p))).ToArray();
        if (next.Length == 0) {
            display((c, r, p));
            display(next.Count());
            break;
        }
        distinct = next.DistinctBy(c => (c.Energy, c.Cost)).ToArray();
    }

    return distinct;
}

## Input

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-23.sample");
var input = File.ReadAllLines(@"day-23.input");

var initial = new List<Amphipod>();

foreach (var d in doors) {
    var c = (int)(input[2][d+1] - 'A' + 1);
    var p = new Location(d, 1);
    var a = new Amphipod(c, p);
    initial.Add(a);
    c = (int)(input[3][d+1] - 'A' + 1);
    p = new Location(d, 2);
    a = new Amphipod(c, p);
    initial.Add(a);
}

initial.Select(a => $"{a}")

index,value
0,D @2/1
1,D @2/2
2,C @4/1
3,A @4/2
4,B @6/1
5,A @6/2
6,C @8/1
7,B @8/2


## Part 1

What is the least energy required to organize the amphipods?

In [ ]:
roomDepth = 2;
var start = new State(initial.ToArray(), null);
start.ShowCurrent();

,,,,,,,,,,,
  D C B C  
  D A A B  
Energy : 17922  Cost: 0


In [ ]:
var expected = new[] {
    (2, 0, 3), (3, 6, 1), (4, 0, 5), (2, 4, 2), (2, 4, 1), (4, 0, 7), (1, 0, 9), (4, 8, 2), (4, 8, 1), (1, 2, 1)
};

var guided = Guided(start, expected);
foreach (var g in guided) g.ShowCurrent();
// display(guided[0].Current);
// guided[0].Needed().Select(n => $"{n}")

Item1,Item2,Item3
3,6,1


0

,,,B,,,,,,,
  D C , C  
  D A A B  
 | B @6/1 >0/3 $40/20
Energy : 17902  Cost: 40


In [ ]:
var min = Minimum(start);
min?.ShowCurrent();

1000: 3229  2000: 4902  3000: 6037  4000: 7038  5000: 8003  6000: 9030  7000: 9541  8000: 10039  9000: 9576  
10000: 8943  11000: 8281  12000: 7592  13000: 6945  14000: 6445  15000: 6409  16000: 6746  17000: 7091  18000: 7180  
19000: 7504  20000: 7641  21000: 6832  22000: 6730  23000: 6645  24000: 6417  25000: 6534  26000: 6487  27000: 6291  
28000: 5704  29000: 5065  30000: 5872  31000: 6381  32000: 7227  33000: 8219  34000: 9316  35000: 9829  36000: 10083  
37000: 10189  38000: 10456  39000: 10637  40000: 10605  41000: 10739  42000: 10646  43000: 10271  44000: 9766  45000: 9387  
46000: 9196  47000: 8888  48000: 8532  49000: 8051  50000: 7464  51000: 6979  52000: 6359  53000: 5879  54000: 5435  
55000: 5178  56000: 4691  57000: 4339  58000: 3851  59000: 3496  60000: 2833  61000: 2148  62000: 1777  63000: 1772  
64000: 1845  65000: 1872  66000: 2314  67000: 2188  68000: 2031  69000: 1964  ,,,,,,,,,,,
  A B C D  
  A B C D  
 | B @6/1 >0/7 $20/40 | A @6/2 >0/0 $8/3 | C @4/1 >0/5 $200/

## Part 2

As you prepare to give the amphipods your solution, you notice that the diagram they handed you was actually folded up. As you unfold it, you discover an extra part of the diagram.

Between the first and second lines of text that contain amphipod starting positions, insert the following lines:
```
  #D#C#B#A#
  #D#B#A#C#
```

In [ ]:
roomDepth = 4;
var extras = new (int c, int r, int p)[] { (4, 2, 2), (4, 2, 3), (3, 4, 2), (2, 4, 3), (2, 6, 2), (1, 6, 3), (1, 8, 2), (3, 8, 3) };
var row1 = initial.Where(a => a.loc.pos == 1);
var row4 = initial.Where(a => a.loc.pos == 2).Select(a => new Amphipod(a.colour, new Location(a.loc.inRoom, roomDepth)));

var extended = extras.Select(t => new Amphipod(t.c, new Location(t.r, t.p))).Concat(row1).Concat(row4);

start = new State(extended.ToArray(), null);
start.ShowCurrent()

,,,,,,,,,,,
  D C B C  
  D C B A  
  D B A C  
  D A A B  
Energy : 40113  Cost: 0


In [ ]:
var expected = new[] {
    (4, 0, 10), (1, 0, 0), (2, 0, 9), (2, 0, 7), (1, 0, 1), (3, 6, 3), (3, 6, 2), (2, 0, 5),
    (4, 0, 3), (2, 4, 4), (2, 4, 3), (2, 4, 2), (3, 6, 1), (1, 0, 9), (4, 8, 4), (2, 4, 1),
    (4, 8, 4), (4, 8, 3), (4, 8, 2), (1, 2, 3), (1, 2, 2), (1, 2, 1)
};

foreach (var g in Guided(start, expected)) g.ShowCurrent();

Item1,Item2,Item3
1,0,0


0

,,,,,,,,,,D
  , C B C  
  D C B A  
  D B A C  
  D A A B  
 | D @2/1 >0/10 $9000/3000
Energy : 35113  Cost: 9000


In [ ]:
min = Minimum(start);
min?.ShowCurrent();

// 48581 is too high

1000: 3194  2000: 4839  3000: 6280  4000: 7595  5000: 8728  6000: 9793  7000: 10412  8000: 10843  9000: 11103  
10000: 11512  11000: 11994  12000: 12447  13000: 13084  14000: 13577  15000: 13995  16000: 14345  17000: 14465  18000: 14483  
19000: 14336  20000: 14396  21000: 14333  22000: 14095  23000: 13557  24000: 12933  25000: 12223  26000: 11489  27000: 10709  
28000: 10123  29000: 9466  30000: 9010  31000: 8467  32000: 8056  33000: 7494  34000: 7365  35000: 7384  36000: 7063  
37000: 7006  38000: 7065  39000: 7065  40000: 6984  41000: 6795  42000: 6224  43000: 6130  44000: 5781  45000: 5712  
46000: 5625  47000: 5357  48000: 5270  49000: 4735  50000: 4803  51000: 5029  52000: 5709  53000: 6371  54000: 6973  
55000: 7417  56000: 7849  57000: 8118  58000: 8454  59000: 8473  60000: 8556  61000: 8275  62000: 8055  63000: 7704  
64000: 7134  65000: 6592  66000: 6144  67000: 5624  68000: 5142  69000: 4738  70000: 4415  71000: 4299  72000: 4018  
73000: 3821  74000: 3698  75000: 3381  7600